In [1]:
from crewai import Agent, Crew, Process, Task, LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

c:\Users\Aksystems\anaconda3\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\Aksystems\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\Aksystems\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\Aksystems

In [2]:
load_dotenv(".env")

AIML_API_KEY = os.getenv("AIML_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
#print(AIML_API_KEY)
#print(SERPER_API_KEY)

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.aimlapi.com/v1",
    api_key=AIML_API_KEY,    
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Write a one-sentence story about numbers."}]
)

print(response.choices[0].message.content)

In a quiet town where arithmetic was law, only the number eight dared to stand sideways and dream of infinity.


In [5]:
# Create an AIML LLM instance
# This is a wrapper around the OpenAI API for AIML
# It allows you to use AIML models with the same interface as OpenAI models
# The AIML LLM instance is created with the model name, base URL, API key, temperature, and max tokens
# The model name is "gpt-4o", which is a specific AIML model
# The base URL is the endpoint for the AIML API
# The API key is used for authentication with the AIML API  
# The temperature controls the randomness of the output, with lower values producing more deterministic results
# The max tokens parameter limits the length of the generated response
 
  

aiml_llm =LLM(
    model="gpt-4o",
    base_url="https://api.aimlapi.com/v1",
    api_key=AIML_API_KEY,
    temperature=0,
    max_tokens=1000
)


In [6]:
# Define Agent with GPT-4o mini Audio Model
input_collector = Agent(
    role="User Input Collector",
    goal="Gather and clarify user requirements for product search from text or voice input, with a focus on products available in Pakistan.",
    backstory="You are an expert in understanding user needs from various input types and translating them into clear search parameters, ensuring the results are relevant to users in Pakistan.",
    llm=aiml_llm,
    verbose=True
)


In [7]:
from IPython.display import display
import ipywidgets as widgets

upload_widget = widgets.FileUpload(accept='.wav', multiple=False)
display(upload_widget)


FileUpload(value=(), accept='.wav', description='Upload')

In [8]:
# Load environment variables
load_dotenv(".env")

AIML_API_KEY = os.getenv("AIML_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Create an AIML LLM instance
aiml_llm =LLM(
    model="gpt-4o",
    base_url="https://api.aimlapi.com/v1",
    api_key=AIML_API_KEY,
    temperature=0,
    max_tokens=1000
)

# AIML API Details
API_KEY = AIML_API_KEY  # Add your AIML API Key here
BASE_URL = "https://api.aimlapi.com/v1/stt"

# Function to send audio to AIML API for transcription
def transcribe_audio_with_aiml(audio_data):
    url = BASE_URL
    headers = {"Authorization": f"Bearer {API_KEY}"}
    
    # Send audio to AIML for transcription
    files = {"audio": ("audio.wav", audio_data, "audio/wav")}
    data = {"model": "#g1_whisper-large"}  # Model for transcription

    try:
        response = requests.post(url, headers=headers, data=data, files=files)
        response.raise_for_status()  # Raise an error if the status code is 400 or higher
        response_data = response.json()
        # Extract transcript
        transcript = response_data["results"]["channels"][0]["alternatives"][0]["transcript"]
        return transcript
    except requests.exceptions.RequestException as e:
        st.error(f"Error during transcription: {e}")
        return None
    except KeyError as e:
        st.error(f"Error parsing transcription result: {e}")
        return None



In [9]:


# Define Agents
input_collector = Agent(
    role="User Input Collector",
    goal="Gather and clarify user requirements for product search from text or voice input, with a focus on products available in Pakistan.",
    backstory="You are an expert in understanding user needs from various input types and translating them into clear search parameters, ensuring the results are relevant to users in Pakistan.",
    llm=aiml_llm,
    verbose=True
)

# Define  web search Tools
# Tools for specific websites
search_tool = SerperDevTool(api_key=SERPER_API_KEY)

scrape_google = ScrapeWebsiteTool(website_url='https://google.com/')
scrape_amazon = ScrapeWebsiteTool(website_url='https://www.amazon.com/')
scrape_daraz = ScrapeWebsiteTool(website_url='https://www.daraz.pk/')


web_searcher = Agent(
    role="Web Search Specialist",
    goal="Find product listings across Google, Amazon, and Daraz",
    backstory="You are a skilled product search expert who knows how to extract valuable listings from multiple platforms.",
    tools=[search_tool, scrape_google, scrape_amazon, scrape_daraz],
    llm=aiml_llm,
    allow_delegation=False,
    verbose=True
)



analyst = Agent(
    role="Product Comparison Expert",
    goal="Evaluate and compare product listings from different vendors to identify the best option based on quality, price, and reviews.",
    backstory="You are a skilled product analysis expert. Your job is to compare various options available online, understand their features and pricing, and help users make informed decisions by providing pros, cons, and recommendations.",
    llm=aiml_llm,
    verbose=True
)

review_tool = WebsiteSearchTool(
    config={
        "llm": {
            "provider": "openai",  # LiteLLM-compatible wrapper
            "config": {
                "model": "gpt-4o",
                "api_key": AIML_API_KEY,
                "base_url": "https://api.aimlapi.com/v1",
                "temperature": 0.5,
                "max_tokens": 1000
            }
        },
        "embedder": {
            "provider": "openai",  # keep this for compatibility
            "config": {
                "model": "text-embedding-3-large",
                "api_key": AIML_API_KEY
            }
        }
    }
)


review_agent = Agent(
    role="Review Analyzer",
    goal="Analyze reviews and summarize user sentiment for the top product recommended by the analysis task from the specified vendor.",
    backstory="You analyze reviews from the selected vendor's website (Daraz, Amazon, AliExpress) for the chosen product to extract pros, cons, and overall user sentiment.",
    tools=[review_tool],
    llm=aiml_llm,
    verbose=True
)

# Define the final recommendation agent
recommender = Agent(
    role="Shopping Recommendation Specialist",
    goal="Recommend the best product based on analysis and user preferences",
    backstory="You provide tailored product recommendations with clear reasoning, combining product details and customer reviews to help users make the best purchase decision.",
    llm=aiml_llm,
    verbose=True
)

In [10]:
# Define Tasks
filters = {
    "min_rating": 4.0,
    "brand": "Sony"
}

# Manually format the filters
brand = filters["brand"] if filters["brand"] else "Any"

description = (
    f"Process the user input: '{{user_input}}'\n"
    f"Use the following filters if applicable:\n"
    f"- Minimum Rating: {filters['min_rating']}\n"
    f"- Preferred Brand: {brand}\n"
    "Generate a refined product search query based on these inputs."
)

# Now, use the formatted description in your Task
input_task = Task(
    description=description,
    expected_output="A well-formed product search query based on the user's input and filter if any .",
    agent=input_collector
)


search_task = Task(
    description="""
        Search online for the best matching products using the refined search query.
        Look for product listings across Google, Amazon, and Daraz. Use appropriate tools for each platform 
        (e.g., Serper API for Google, scraping for Amazon and Daraz).

        Return a JSON list of the **top 3 products** with the following fields:
        - name (title)
        - price
        - rating
        - url
        - image_url
        - source (e.g., Amazon, Daraz, etc.)
    """,
    expected_output="""
        A JSON-formatted list of 3 product listings from Google, Amazon, and Daraz.
        Each product must include: name, price, rating, url, image_url, and source.
    """,
    agent=web_searcher,
    context=[input_task]
)


analysis_task = Task(
    description=(
        "Analyze the structured product listings (JSON format) from different websites (Google, Amazon, Daraz). "
        "Compare features, price, rating, and source for each. "
        "Rank the top 3 products based on overall value (considering quality, affordability, rating). "
        "For each of the top 3, return: name, price, rating, source, brief reason for ranking."
    ),
    expected_output="""
        A ranked list (1 to 3) of the top product recommendations.
        Each entry should include name, price, rating, source, and a short reason why it was selected.
    """,
    agent=analyst,
    context=[search_task]
)



review_task = Task(
    description=(
        "Using the top product recommendation and vendor from the analysis task, "
        "summarize customer reviews for this product. Review feedback will include pros, cons, and sentiment analysis. "
        "First, determine the correct website URL based on the vendor (Amazon, Daraz, or AliExpress). "
        "Then use the WebsiteSearchTool to analyze reviews from that specific website."
    ),
    expected_output="A summarized list of pros, cons, and user sentiment for the selected product from the appropriate vendor's website.",
    agent=review_agent,
    context=[analysis_task]
)

recommendation_task = Task(
    description=(
        "Provide a final product recommendation summary based on the top 3 ranked products and their customer reviews. "
        "Summarize key features, pros/cons, and customer sentiment for each. "
        "Highlight which one is the best choice and why, but present all three options with image URLs."
    ),
    expected_output="""
        A summary of top 3 recommended products.
        For each: name, price, rating, image_url, pros/cons, sentiment, and final verdict on the best one.
    """,
    agent=recommender,
    context=[
        analysis_task,
        review_task
    ]
)


In [11]:
product_knowledge = StringKnowledgeSource(
    content="Information about current product trends, including electronics, fashion, beauty, lifestyle, and more."
)

In [12]:
# 🤖 Shopping Crew Setup
shopping_crew = Crew(
    agents=[input_collector, web_searcher, analyst, review_agent, recommender],
    tasks=[input_task, search_task, analysis_task, review_task, recommendation_task],
    verbose=True,
    process=Process.sequential,
    embedder={
        "provider": "aimlapi",
        "config": {
            "model": "text-embedding-3-large",
            "api_key": AIML_API_KEY,
            }
         }
    )


# This will generate an interactive HTML plot of your flow
#shopping_crew.plot("shop_smart_flow.html")



In [13]:
# Replace Streamlit functions with standard Python input for testing in Jupyter

# Choose input type (simulate user choice between Text and Voice)
input_type = input("Choose input type (Text/Voice): ").strip()

# Get user input directly for text input
if input_type.lower() == "text":
    user_input = input("Please enter your product query: ")
elif input_type.lower() == "voice":
    # Simulate voice input (you can replace this with a real voice function for actual testing)
    user_input = "Simulated voice input: search for laptops" 
else:
    user_input = None
    print("Invalid input type selected. Please try again.")

# If input is provided, process the input using Crew AI
if user_input:
    # Simulate calling the Crew AI and kicking off the tasks
    result = shopping_crew.kickoff(inputs={"user_input": user_input})

    # Output the result
    print("Processed result:", result.raw)
else:
    print("No input received. Please try again.")


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 65a7b0a3-2eaf-4272-bfde-cb996fe0ccf2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e3099217-3bda-4bf4-89e4-be524d6ccc0d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e3099217-3bda-4bf4-89e4-be524d6ccc0d
       Status: Executing Task...
    └── 🤖 Agent: User Input Collector
            Status: In Progress

# Agent: User Input Collector
## Task: Process the user input: 'help me find phone under 300 $'
Use the following filters if applicable:
- Minimum Rating: 4.0
- Preferred Brand: Sony
Generate a refined product search query based on these inputs.


🤖 Agent: User Input Collector
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: e3099217-3bda-4bf4-89e4-be524d6ccc0d
│      Assigned to: User Input Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: User Input Collector
│           Status: ✅ Completed
├── 📋 Task: ad48c23c-e13b-424e-95ee-097be4b164c3
│      Assigned to: Web Search Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Search Specialist
│           Status: ✅ Completed
├── 📋 Task: 0b2120b6-5ca1-4f9b-87b2-663522b899d1
│      Assigned to: Product Comparison Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Product Comparison Expert
│           Status: ✅ Completed
├── 📋 Task: 421dce2f-ddc0-4d9f-8b20-c126c8c0f4e3
│      Assigned to: Review Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Review Analyzer
│           Status: ✅ Completed
└── 📋 Task: e779e8a0-6c04-403b-a197-b3fbd7c0ed61
       Status: Executing Task...
    └── 🤖 Agent: Shopping Recommendation Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: e3099217-3bda-4bf4-89e4-be524d6ccc0d
│      Assigned to: User Input Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: User Input Collector
│           Status: ✅ Completed
├── 📋 Task: ad48c23c-e13b-424e-95ee-097be4b164c3
│      Assigned to: Web Search Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Search Specialist
│           Status: ✅ Completed
├── 📋 Task: 0b2120b6-5ca1-4f9b-87b2-663522b899d1
│      Assigned to: Product Comparison Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Product Comparison Expert
│           Status: ✅ Completed
├── 📋 Task: 421dce2f-ddc0-4d9f-8b20-c126c8c0f4e3
│      Assigned to: Review Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Review Analyzer
│           Status: ✅ Completed
└── 📋 Task: e779e8a0-6c04-403b-a197-b3fbd7c0ed61
       Assigned to: Shopping Recommendation Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Shopping Recommendation Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e779e8a0-6c04-403b-a197-b3fbd7c0ed61                                                                     │
│  Agent: Shopping Recommendation Specialist                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 65a7b0a3-2eaf-4272-bfde-cb996fe0ccf2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processed result: **1. Sony Xperia 10V 5G + 4G LTE (128GB + 8GB) 6.1" Factory Unlocked**
   - **Price:** $268
   - **Rating:** 4.0
   - **Image URL:** [Sony Xperia 10V Image](https://example.com/sony-xperia-10v-image)
   - **Pros:**
     - Modern features including 5G connectivity.
     - Large storage capacity of 128GB and 8GB RAM for enhanced performance.
     - Reasonable price for the features offered.
   - **Cons:**
     - Some users may find the design less appealing compared to other premium models.
     - Battery life could be better according to some reviews.
   - **Customer Sentiment:** Generally positive, with users appreciating the performance and future-proofing features. Some concerns about battery life and design aesthetics.
   - **Final Verdict:** This is the best choice for users looking for a modern smartphone with advanced features at a reasonable price. Its 5G capability and large storage make it a future-proof option.

**2. Sony Xperia XZ Premium - Unlocked Smartph